In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

In [ ]:
data = pd.read_csv('california_housing.csv')

In [ ]:
data.head()

In [ ]:
import pymc as pm #pip install pymc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
# Feature (X) and target vector (y)
X = data.drop('median_house_value', axis=1)
y = data['median_house_value']

In [ ]:
# Focus on one feature for clear visualization (e.g., RM)
X = X[['total_rooms']]  # average rooms per dwelling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled,
                                                    y,
                                                    test_size=0.2, 
                                                    random_state=42)

In [ ]:
# Bayesian Lasso Model (Laplace priors)
with pm.Model() as bl_model:
    w = pm.Laplace("w", mu=0, b=1) #it is slope in regression and why laplace is due to L1, Mu=mean,b=std
    b = pm.Normal("b", mu=0, sigma=10) #it is normal intercept, sigma-noise or error
    sigma = pm.HalfCauchy("sigma", beta=1) #half-cauchy is distribution which will tend towards positive

    mu = w * X_train.flatten() + b # mu = w * X + b
    y_obs = pm.Normal("y_obs", mu=mu, sigma=sigma, observed=y_train) #y∼N(μ,σ) #Likelihood Function

    trace = pm.sample(2000, tune=1000, target_accept=0.9, random_seed=42)
    #all the samples drawn from the posterior distribution of your model parameters.

In [ ]:
#Posterior Predictive Samples
X_seq = np.linspace(X_train.min(), X_train.max(), 100)
with bl_model:
    post_pred = pm.sample_posterior_predictive(
        trace,
        var_names=["w", "b", "sigma"],
        predictions=True
    )

#### Markov Chains can be understood as a process of moving step-by-step through states where the choice of the next state depends only on the current state and the probability distribution of possible next states.

In [ ]:
# Compute mean and credible intervals
# Markov Chain Monte Carlo (MCMC) sampler
posterior_means = trace.posterior["w"].mean(("chain", "draw")).values
posterior_intercept = trace.posterior["b"].mean(("chain", "draw")).values
pred_mean = posterior_means * X_seq + posterior_intercept

In [ ]:
# Extract 95% credible interval for visualization
pred_samples = np.array([
    (trace.posterior["w"].values.flatten() * x + trace.posterior["b"].values.flatten())
    for x in X_seq
])
lower = np.percentile(pred_samples, 2.5, axis=1)
upper = np.percentile(pred_samples, 97.5, axis=1)

In [ ]:
# Visualization
plt.figure(figsize=(10, 6))
plt.scatter(X_train, y_train, color='gray', alpha=0.5, label='Training data')
plt.plot(X_seq, pred_mean, color='red', label='Posterior Mean Prediction')
plt.fill_between(X_seq.flatten(), lower, upper, color='orange', alpha=0.3, label='95% Credible Interval')

plt.title("Bayesian Lasso Regression – 95% Credible Intervals")
plt.xlabel("total_rooms")
plt.ylabel("median_house_value")
plt.legend()
plt.show()